## A fast and dirty intro to running a model via web service

Have you ever needed to score data with a predictive model in (near) real time, using a remote server/machine? While not a statistical or machine learning topic, it's one that I struggled with for a while, and haven't found many resources for.

In this post, we're going to assume an understanding of predictive modeling and python, to focus on a simple way to set up a web service that can recieve data and return model predictions over the internet.

This way is by no means definitive, but is straightforward and effective.

___
### [Part 2: Listener](http://spencercarter.info/docs/model_web_service_flask_2.ipynb)

Okay, now we've got a model. To get out web service going, we're going to need two things:
1. A scoring function to turn data to predictions
2. A web service running on our local machine, listening for requests

#### 2.1: Scoring Function
Let's start with the scoring function. This function is going to assume that the data will come in as a JSON, which we're just going to use to pass data in the form:

```
{<variable1>:<value>, <variable2>:<value>, ... , <variableN>:<value>}
```

This should look familiar; simple JSONs look like a dict in python. Just about any data structure we may want to impose will work here, but I've chosen JSON because it's easy to receive in the web service with *flask.request*.get_json(), and pandas makes it simple to send/score with *DataFrame*.to_json() and *pandas*.read_json().

To ensure variables are consistent with expectations, we're going to pass all the data recieved, but keep just the values that the model expects, and type them accordingly (object, int, float, etc.).

In [1]:
# Repeat the setup block for the listener and sender
import sklearn as sk
import numpy as np
from sklearn.datasets import load_boston
from sklearn.linear_model import LassoCV
import pandas as pd
from flask import Flask
from flask import request
import requests
import pickle
from time import time

def unpickle_me(infile):
    with open(infile, 'rb') as f:
        unpickled = pickle.load(f)
    return unpickled

# Read in our pickled dictionary
stored = unpickle_me('model_web_service.pickle')

# Read our data and model back into memory
df_types = stored['dtypes']
predictors = stored['predictors']
X = stored['X']
lasso = stored['lasso']

Let's take a look at what the JSON data will look like. Call *DataFrame*.to_json() on a slice of a DF.

In [2]:
X.iloc[0].to_json()

'{"crim":0.00632,"zn":18.0,"indus":2.31,"chas":0.0,"nox":0.538,"rm":6.575,"age":65.2,"dis":4.09,"rad":1.0,"tax":296.0,"ptratio":15.3,"b":396.9,"lstat":4.98}'

Next, create a scoring function that takes a JSON, and returns a model prediction.

In [3]:
def score_obs(indata, model=lasso, predictors=predictors, df_types=df_types):
    """
    Scoring function. Takes a JSON of variables and values, then runs them through the model and returns a prediction
    """

    # Structure the input into a DF, but then impose the expected variable structure, using the .loc slicer
    data = pd.read_json(indata,typ='series').to_frame().T.loc[:,predictors]
    
    # .loc will drop excess, and create variables that aren't present. Fill their values so the model runs
    data.fillna(0, inplace=True) # !!! In practice, you should actually build in error handling or imputation
    
    # Fix the dtypes
    for c in data:
        data[c] = data[c].astype(df_types[c])
    
    # predict returns an array, so grab the first value
    return model.predict(data.values.reshape(1,-1))[0]

# To make sure everything is working, compare a direct prediction against one from the function
print("Direct prediction: {} \nUsing this function: {}".format(lasso.predict(X.iloc[0:1])[0],score_obs(X.iloc[0].to_json())))

Direct prediction: 30.505530469875787 
Using this function: 30.505530469875787


*Note on some hangups I've run into:*

As you can see in the above, to_json() returns a string. To read a single observation with read_json(), we need to read it as a series, shaped (p,)

```python
pd.read_json(indata,typ='series')
```

To get back to the intended structure, we send that series to a DataFrame, and transpose to get the (1,p) shape.

```python
.to_frame().T
```
Now the model is in memory, and we have a scoring function! Let's set up the web service.

___
#### 2.2: Web Service

This is a simple [RESTful](https://en.wikipedia.org/wiki/Representational_state_transfer) webservice that accepts data via POST, and returns the model prediction.

*Strictly speaking, model predictions may more closely match GET requests in REST, because they can be repeated without care and don't alter anything on the server (idempotent). However, I'm using POST because model data can get extremely lengthy, and might exceed what some broswers can handle in a URL. [Here's some more info about REST](http://blog.teamtreehouse.com/the-definitive-guide-to-get-vs-post).*

In [6]:
app = Flask(__name__)

app.silent = True # Suppressess logging and errors. In practice, you'll likely want them, or import logging

@app.route("/mass", methods=['POST'])
def predict_medv():
    
    # Get the JSON from the request and send to our scoring function
    score = score_obs(request.get_json()) 
    
    return "{0:0.2f}".format(score)

app.run(port=1234) # defaults to localhost. Use host= option to change

Let's break it down:

```python
@app.route("/mass", methods=['POST'])
```

Configured the the web service to run at ```<host>:<port>/mass``` and specifies that we'll be sending POST requests.

```python
def predict_medv():
```

Creates the function that will execute upon requests. And lastly,

```python
app.run(port=1234)
```

Starts up the web service on the ```<host>:<port>``` specified, defaulting host to localhost (127.0.0.1).

At this point, the web service is up, and will keep the shell running, so if you're in a notebook, this will lock it up. In the raw code, I've broken out the listener from the sender so you can run both at the same time.